In [2]:
# Code for ETL operations on Bank data
# Importing the required libraries
import requests                     #La bibliothèque utilisée pour accéder aux informations à partir de l’URL.
from bs4 import BeautifulSoup       # La bibliothèque contenant la fonction utilisée pour le webscraping.BeautifulSoup
import pandas as pd                 # La bibliothèque utilisée pour traiter les données extraites, les stocker dans les formats requis et communiquer avec les bases de données.
import sqlite3                      # Bibliothèque requise pour créer une connexion au serveur de base de données.
import numpy as np                  # La bibliothèque nécessaire à l’opération d’arrondi mathématique telle qu’elle est requise dans les objectifs.
from datetime import datetime       # La bibliothèque contenant la fonction utilisée pour extraire l’horodatage à des fins de journalisation.datetime

In [3]:

# Preliminaries: Installing libraries and downloading data
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = ["Name", "MC_USD_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'

In [4]:
page = requests.get(url).text
data = BeautifulSoup(page,'html.parser')
df = pd.DataFrame(columns=table_attribs)
tables = data.find_all('tbody')

In [5]:
rows = tables[0].find_all('tr')
row = rows[2]
col = row.find_all('td')
col

[<td>2
 </td>,
 <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/https://en.wikipedia.org/wiki/Bank_of_America" title="Bank of America">Bank of America</a>
 </td>,
 <td>231.52
 </td>]

In [ ]:
# Task 2 : Extraction of data
def extract(url, table_attribs):
    page = requests.get(url).text  #Downloads the content of the webpage from the provided URL
    data = BeautifulSoup(page, 'html.parser')  # Uses BeautifulSoup to parse the HTML content of the page
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')            # Searches for all tables in the page (specifically, <tbody> tags)
    rows = tables[0].find_all('tr')            # Finds all rows (tr tags) in the first table found

    for row in rows:
        col = row.find_all('td')               # Retrieves all columns (td tags) in the row
        if len(col) != 0:                      # Checks if the row contains any columns
            if col[1].find('a') is not None:   # Checks if the second column contains an anchor (<a>) tag
                l = len(col[1].find_all('a'))  # Gets the total number of links in the second column
                # Creates a dictionary with:
                # - "Name": the text of the last link found in the second column, stripped of any extra spaces
                # - "MC_USD_Billion": the content of the third column, converted to a float
                data_dict = {
                    "Name": [col[1].find_all('a')[l-1].text.strip()],
                    "MC_USD_Billion": [float(col[2].contents[0].strip())]
                }
            df1 = pd.DataFrame(data_dict)
            df = pd.concat([df, df1], ignore_index=True)
    return df
df = extract(url, table_attribs)
df

In [13]:
df_exchange_rate = pd.read_csv("exchange_rate.csv")                         # Read exchage data
exchange_rate = df_exchange_rate.set_index('Currency').to_dict()['Rate']    # Convert to dict

df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

In [14]:
df['MC_EUR_Billion'][4]

146.86